In [70]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import numpy as np

from linearmodels.panel import PanelOLS
import datetime as dt

# Characteristcs OLS

Professor said we should start with a regular OLS Regression.
I quickly looked into this and think a random effects panel regression would probably fit our data better (still need to read more into this)
So probably we can first do a regular OLS and than compare it to the random effects panel regression. Just some initial thoughts though!!!
Here is some intial code to run them:


In [71]:
#decide which df to use
df = pd.read_csv("Dataframes/characteristics.csv")
#df = df.loc[:, ["Instrument", "Date", "Earnings Per Share - Actual Surprise", "Revenue - Actual", "Net Income after Tax"]]

df

,Instrument,Date,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),loss firm status
0,POOL.OQ,2022-10-01,4.196,1.615339e+09,1.406160e+10,1.259813e+10,2.383926,-1
1,POOL.OQ,2022-07-01,1.503,2.055818e+09,1.556187e+10,1.405795e+10,2.749481,-1
2,POOL.OQ,2022-04-01,34.342,1.412650e+09,1.843682e+10,1.696712e+10,3.422880,-1
3,POOL.OQ,2022-01-01,40.267,1.035557e+09,2.384882e+10,2.268979e+10,4.504382,-1
4,POOL.OQ,2021-10-01,17.194,1.411448e+09,1.769954e+10,1.742020e+10,3.684400,-1
...,...,...,...,...,...,...,...,...
2485,AVY.N,2022-10-01,-0.131,2.317100e+09,1.622502e+10,1.322042e+10,1.819581,-1
2486,AVY.N,2022-07-01,11.635,2.347000e+09,1.650430e+10,1.343710e+10,1.877800,-1
2487,AVY.N,2022-04-01,10.595,2.349300e+09,1.753708e+10,1.441548e+10,2.053846,-1
2488,AVY.N,2022-01-01,0.294,2.183200e+09,2.087304e+10,1.793104e+10,2.546597,-1


#### OLS Regression

In [72]:
#identifying outliers and replacing them with NA
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7 #1.5 is standard threshold but we still want to keep enough variation in the data so setting threshol higher here
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_accuracy_new = df.copy()
df_accuracy_new["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_accuracy_new = df_accuracy_new.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_accuracy_new["Recommendation - Mean (1-5)"] = df_accuracy_new["Recommendation - Mean (1-5)"].fillna(0)
df_accuracy_new = df_accuracy_new.dropna()
df_accuracy_new.value_counts("loss firm status")


loss firm status
-1    2267
 1     138
dtype: int64

In [73]:
df_accuracy_new

,Instrument,Date,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),loss firm status
0,POOL.OQ,2022-10-01,4.196,1.615339e+09,1.406160e+10,1.259813e+10,2.383926,-1
1,POOL.OQ,2022-07-01,1.503,2.055818e+09,1.556187e+10,1.405795e+10,2.749481,-1
2,POOL.OQ,2022-04-01,34.342,1.412650e+09,1.843682e+10,1.696712e+10,3.422880,-1
3,POOL.OQ,2022-01-01,40.267,1.035557e+09,2.384882e+10,2.268979e+10,4.504382,-1
4,POOL.OQ,2021-10-01,17.194,1.411448e+09,1.769954e+10,1.742020e+10,3.684400,-1
...,...,...,...,...,...,...,...,...
2485,AVY.N,2022-10-01,-0.131,2.317100e+09,1.622502e+10,1.322042e+10,1.819581,-1
2486,AVY.N,2022-07-01,11.635,2.347000e+09,1.650430e+10,1.343710e+10,1.877800,-1
2487,AVY.N,2022-04-01,10.595,2.349300e+09,1.753708e+10,1.441548e+10,2.053846,-1
2488,AVY.N,2022-01-01,0.294,2.183200e+09,2.087304e+10,1.793104e+10,2.546597,-1


In [75]:
rescale = df_accuracy_new
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["Revenue - Actual"] = MinMaxScaler().fit_transform(np.array(rescale["Revenue - Actual"]).reshape(-1,1))
rescale["Enterprise Value"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value"]).reshape(-1,1))
rescale["Market Capitalization"] = MinMaxScaler().fit_transform(np.array(rescale["Market Capitalization"]).reshape(-1,1))
rescale["Enterprise Value To Sales (Daily Time Series Ratio)"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value To Sales (Daily Time Series Ratio)"]).reshape(-1,1))

rescale

,Instrument,Date,Earnings Per Share - Actual Surprise,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),loss firm status
0,POOL.OQ,2022-10-01,4.196,0.010538,0.026228,0.002749,0.029064,-1
1,POOL.OQ,2022-07-01,1.503,0.013420,0.026725,0.003255,0.029495,-1
2,POOL.OQ,2022-04-01,34.342,0.009212,0.027679,0.004262,0.030290,-1
3,POOL.OQ,2022-01-01,40.267,0.006746,0.029473,0.006244,0.031566,-1
4,POOL.OQ,2021-10-01,17.194,0.009205,0.027434,0.004419,0.030598,-1
...,...,...,...,...,...,...,...,...
2485,AVY.N,2022-10-01,-0.131,0.015129,0.026945,0.002964,0.028398,-1
2486,AVY.N,2022-07-01,11.635,0.015325,0.027038,0.003040,0.028467,-1
2487,AVY.N,2022-04-01,10.595,0.015340,0.027380,0.003378,0.028675,-1
2488,AVY.N,2022-01-01,0.294,0.014253,0.028486,0.004596,0.029256,-1


In [76]:
y = rescale["Earnings Per Share - Actual Surprise"]
#X = df_accuracy_new[['Revenue - Actual', 'Net Income after Tax']]
X = rescale.iloc[:, 3:]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     OLS Regression Results                                     
================================================================================================
Dep. Variable:     Earnings Per Share - Actual Surprise   R-squared:                       0.007
Model:                                              OLS   Adj. R-squared:                  0.005
Method:                                   Least Squares   F-statistic:                     3.358
Date:                                  Fri, 24 Feb 2023   Prob (F-statistic):            0.00501
Time:                                          18:41:34   Log-Likelihood:                -10155.
No. Observations:                                  2405   AIC:                         2.032e+04
Df Residuals:                                      2399   BIC:                         2.036e+04
Df Model:                                             5                                         
Covariance Type:                              nonrobust                                         
=======================================================================================================================
                                                          coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------
const                                                   3.7564      1.285      2.924      0.003       1.237       6.275
Revenue - Actual                                        6.0740      4.358      1.394      0.164      -2.472      14.620
Enterprise Value                                       61.9606     33.844      1.831      0.067      -4.406     128.327
Market Capitalization                                 -65.2560     31.830     -2.050      0.040    -127.674      -2.838
Enterprise Value To Sales (Daily Time Series Ratio)   -14.8499     13.901     -1.068      0.286     -42.109      12.409
loss firm status                                       -2.0790      0.733     -2.835      0.005      -3.517      -0.641
==============================================================================
Omnibus:                      441.668   Durbin-Watson:                   1.700
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3074.177
Skew:                           0.682   Prob(JB):                         0.00
Kurtosis:                       8.368   Cond. No.                         189.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### Random Effects Panel Regression

In [77]:
df_accuracy_new['Date'] = pd.to_datetime(df_accuracy_new['Date'], infer_datetime_format=True)
df_accuracy_new.dtypes

Instrument                                                     object
Date                                                   datetime64[ns]
Earnings Per Share - Actual Surprise                          float64
Revenue - Actual                                              float64
Enterprise Value                                              float64
Market Capitalization                                         float64
Enterprise Value To Sales (Daily Time Series Ratio)           float64
loss firm status                                                int64
dtype: object

In [78]:
df_accuracy_new.set_index(["Instrument", "Date"], inplace=True)
df_accuracy_new

Earnings Per Share - Actual Surprise  Revenue - Actual  \
Instrument Date                                                                 
POOL.OQ    2022-10-01                                 4.196          0.010538   
           2022-07-01                                 1.503          0.013420   
           2022-04-01                                34.342          0.009212   
           2022-01-01                                40.267          0.006746   
           2021-10-01                                17.194          0.009205   
...                                                     ...               ...   
AVY.N      2022-10-01                                -0.131          0.015129   
           2022-07-01                                11.635          0.015325   
           2022-04-01                                10.595          0.015340   
           2022-01-01                                 0.294          0.014253   
           2021-10-01                                 4.784          0.013524   

                       Enterprise Value  Market Capitalization  \
Instrument Date                                                  
POOL.OQ    2022-10-01          0.026228               0.002749   
           2022-07-01          0.026725               0.003255   
           2022-04-01          0.027679               0.004262   
           2022-01-01          0.029473               0.006244   
           2021-10-01          0.027434               0.004419   
...                                 ...                    ...   
AVY.N      2022-10-01          0.026945               0.002964   
           2022-07-01          0.027038               0.003040   
           2022-04-01          0.027380               0.003378   
           2022-01-01          0.028486               0.004596   
           2021-10-01          0.028319               0.004404   

                       Enterprise Value To Sales (Daily Time Series Ratio)  \
Instrument Date                                                              
POOL.OQ    2022-10-01                                           0.029064     
           2022-07-01                                           0.029495     
           2022-04-01                                           0.030290     
           2022-01-01                                           0.031566     
           2021-10-01                                           0.030598     
...                                                                  ...     
AVY.N      2022-10-01                                           0.028398     
           2022-07-01                                           0.028467     
           2022-04-01                                           0.028675     
           2022-01-01                                           0.029256     
           2021-10-01                                           0.029081     

                       loss firm status  
Instrument Date                          
POOL.OQ    2022-10-01                -1  
           2022-07-01                -1  
           2022-04-01                -1  
           2022-01-01                -1  
           2021-10-01                -1  
...                                 ...  
AVY.N      2022-10-01                -1  
           2022-07-01                -1  
           2022-04-01                -1  
           2022-01-01                -1  
           2021-10-01                -1  

[2405 rows x 6 columns]

In [79]:
import pandas as pd
import scipy.stats as st
import statsmodels.api as sm
import statsmodels.graphics.tsaplots as tsap
from statsmodels.compat import lzip
from statsmodels.stats.diagnostic import het_white
from matplotlib import pyplot as plt

In [80]:
y_var_name = 'Earnings Per Share - Actual Surprise'
X_var_names = ["Revenue - Actual","Enterprise Value","Market Capitalization","loss firm status", "Enterprise Value To Sales (Daily Time Series Ratio)"]
pooled_y=df_accuracy_new[y_var_name]
pooled_X=df_accuracy_new[X_var_names]

In [81]:
pooled_X = sm.add_constant(pooled_X)
pooled_X

const  Revenue - Actual  Enterprise Value  \
Instrument Date                                                    
POOL.OQ    2022-10-01    1.0          0.010538          0.026228   
           2022-07-01    1.0          0.013420          0.026725   
           2022-04-01    1.0          0.009212          0.027679   
           2022-01-01    1.0          0.006746          0.029473   
           2021-10-01    1.0          0.009205          0.027434   
...                      ...               ...               ...   
AVY.N      2022-10-01    1.0          0.015129          0.026945   
           2022-07-01    1.0          0.015325          0.027038   
           2022-04-01    1.0          0.015340          0.027380   
           2022-01-01    1.0          0.014253          0.028486   
           2021-10-01    1.0          0.013524          0.028319   

                       Market Capitalization  loss firm status  \
Instrument Date                                                  
POOL.OQ    2022-10-01               0.002749                -1   
           2022-07-01               0.003255                -1   
           2022-04-01               0.004262                -1   
           2022-01-01               0.006244                -1   
           2021-10-01               0.004419                -1   
...                                      ...               ...   
AVY.N      2022-10-01               0.002964                -1   
           2022-07-01               0.003040                -1   
           2022-04-01               0.003378                -1   
           2022-01-01               0.004596                -1   
           2021-10-01               0.004404                -1   

                       Enterprise Value To Sales (Daily Time Series Ratio)  
Instrument Date                                                             
POOL.OQ    2022-10-01                                           0.029064    
           2022-07-01                                           0.029495    
           2022-04-01                                           0.030290    
           2022-01-01                                           0.031566    
           2021-10-01                                           0.030598    
...                                                                  ...    
AVY.N      2022-10-01                                           0.028398    
           2022-07-01                                           0.028467    
           2022-04-01                                           0.028675    
           2022-01-01                                           0.029256    
           2021-10-01                                           0.029081    

[2405 rows x 6 columns]

In [82]:
pooled_olsr_model = sm.OLS(endog=pooled_y, exog=pooled_X)

In [83]:
pooled_olsr_model_results = pooled_olsr_model.fit()

In [84]:
print(pooled_olsr_model_results.summary())

                                     OLS Regression Results                                     
Dep. Variable:     Earnings Per Share - Actual Surprise   R-squared:                       0.007
Model:                                              OLS   Adj. R-squared:                  0.005
Method:                                   Least Squares   F-statistic:                     3.358
Date:                                  Fri, 24 Feb 2023   Prob (F-statistic):            0.00501
Time:                                          18:41:59   Log-Likelihood:                -10155.
No. Observations:                                  2405   AIC:                         2.032e+04
Df Residuals:                                      2399   BIC:                         2.036e+04
Df Model:                                             5                                         
Covariance Type:                              nonrobust                                         
                              